#### Import Statements

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

import tqdm
import matplotlib.pyplot as plt 

##### Parameters Function

In [ ]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
# Download data
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [ ]:
batch_size=32

# Create data loader
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# Load one batch of training data
for data in train_dataloader:
  break

  x = data[0]
  y = data[1]
  print("Shape of x [N, C, H, W]:", x.shape)
  print("Shape of y:", y.shape)

  plt.figure(figsize=(8,10))
  for i in range(25):
    plt.subplot(5,5,i+1)
    plt.imshow(x[i,0,:,:], cmap="gray")
    plt.title("Label:%i" %y[i])
    

#### LeNet Model

In [ ]:
class LeNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(in_channels=1, 
                  out_channels=6, 
                  kernel_size=5, 
                  stride=1, 
                  padding=2),
        nn.Sigmoid(),
        nn.AvgPool2d(kernel_size=2,
                     stride=2)
    )
    self.conv2 = nn.Sequential(
        nn.Conv2d(in_channels=6,  # This number must be the same as the output of the previous layer
                  out_channels=16, 
                  kernel_size=5, 
                  stride=1, 
                  padding=0),
        nn.Sigmoid(), #ReLU is better
        nn.AvgPool2d(kernel_size=2,
                     stride=2)
    )
    self.flatten = nn.Flatten()

    self.fc1 = nn.Sequential(
        nn.Linear(16*5*5, 120),
        nn.Sigmoid()
    )
    
    self.fc2 = nn.Sequential(
        nn.Linear(120, 84),
        nn.Sigmoid()
    )

    self.fc3 = nn.Linear(84,10)

  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.fc2(x)
    logits = self.fc3(x)
    return logits


##### Train Function

In [ ]:
def train(dataloader, model, loss_fn, optimizer, device):
  model.train() # set model to train model
  for step, (x, y) in enumerate(dataloader): 
    # send data to GPU or CPU
    x = x.to(device)
    y = y.to(device)
    
    # feed the data to the model
    pred = model(x)
    
    # compute the loss
    loss = loss_fn(pred, y)
    
    # backpropagation (update the parameters)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if step % 200 == 0: 
      loss  = loss.item()
      print('Current Step: %d, loss:%.4f' %(step, loss))

##### Test Function

In [ ]:
def test(dataloader, model, loss_fn, device):
  num_batch = len(dataloader)

  model.eval()
  test_loss = 0
  correct = 0

  with torch.no_grad():
    for x, y in dataloader:
      x = x.to(device)
      y = y.to(device)
      pred = model(x)
      loss = loss_fn(pred, y)
      test_loss += loss.item()
      
      y_hat = pred.argmax(1)
      correct_batch = (y_hat == y).type(torch.float).sum().item()
      correct += correct_batch
  test_loss /= num_batch
  correct = correct / (num_batch * batch_size)

  print("Test Accuracy:%.4f" % correct)

#### Train the Model

In [ ]:
# Get CPU or GPU for the training
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create the model
model = LeNet().to(device)
print("---------------\nTraining the LeNet model")
print("Total number of trainable parameters:%d \n------------" %count_parameters(model))
print(model)

# Optimizing the model parameter
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

# Train model in epochs
epochs = 5
for t in tqdm.tqdm(range(epochs)):
  print('Epoch %d \n----------------' %t)
  train(train_dataloader, model, loss_fn, optimizer, device)
  test(test_dataloader, model, loss_fn, device)
print("Done!")


---------------
Training the LeNet model
Total number of trainable parameters:61706 
------------
LeNet(
  (conv1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Sigmoid()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (conv2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): Sigmoid()
  )
  (fc2): Sequential(
    (0): Linear(in_features=120, out_features=84, bias=True)
    (1): Sigmoid()
  )
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 0 
----------------
Current Step: 0, loss:2.3852
Current Step: 200, loss:2.3089
Current Step: 400, loss:2.3050
Current Step: 600, loss:2.3137
Current Step: 800, loss:2.2859
Current Step: 1000, loss:2.3146
Current Step: 1200, loss:2.3026
Current Step: 1400, loss:2.2905
Current Step: 1600, loss:2.2961
Current Step: 1800, loss:2.3108


 20%|██        | 1/5 [00:26<01:44, 26.18s/it]

Test Accuracy:0.0998
Epoch 1 
----------------
Current Step: 0, loss:2.2929
Current Step: 200, loss:2.3089
Current Step: 400, loss:2.3049
Current Step: 600, loss:2.3137
Current Step: 800, loss:2.2861
Current Step: 1000, loss:2.3146
Current Step: 1200, loss:2.3026
Current Step: 1400, loss:2.2906
Current Step: 1600, loss:2.2961
Current Step: 1800, loss:2.3107


...

Test Accuracy:0.0998
Epoch 3 
----------------
Current Step: 0, loss:2.2929
Current Step: 200, loss:2.3089
Current Step: 400, loss:2.3047
Current Step: 600, loss:2.3137
Current Step: 800, loss:2.2864
Current Step: 1000, loss:2.3146
Current Step: 1200, loss:2.3027
Current Step: 1400, loss:2.2908
Current Step: 1600, loss:2.2963
Current Step: 1800, loss:2.3105


 80%|████████  | 4/5 [01:41<00:25, 25.38s/it]

Test Accuracy:0.0998
Epoch 4 
----------------
Current Step: 0, loss:2.2929
Current Step: 200, loss:2.3089
Current Step: 400, loss:2.3046
Current Step: 600, loss:2.3137
Current Step: 800, loss:2.2866
Current Step: 1000, loss:2.3146
Current Step: 1200, loss:2.3027
Current Step: 1400, loss:2.2909
Current Step: 1600, loss:2.2963
Current Step: 1800, loss:2.3105


100%|██████████| 5/5 [02:09<00:00, 25.83s/it]

Test Accuracy:0.0998
Done!
